In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

taxi = pd.DataFrame()

def to_string(a):
    if isinstance(a, float):
        return str(int(a))
    elif isinstance(a, int):
        return str(a)
    else:
        return str(a)

for file in os.listdir('.'):
    if file[-4:] == 'xlsx':
        temp_df = pd.read_excel(file)
        temp_df['Источник'] = [file[:-5] for i in range(temp_df.shape[0])]
        temp_df = temp_df.loc[:, ['Телефон', 'Источник']]
        temp_df = temp_df.drop(temp_df[temp_df['Телефон'].isnull()].index)
        temp_df['Телефон'] = temp_df['Телефон'].apply(to_string)
        taxi = taxi.append(temp_df, ignore_index=True)
 
           
        
pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'


mobil = taxi['Телефон'].to_list()
        

for i, j in enumerate(mobil):
    if type(j) == str:
        mobil[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
        mobil[i] = mobil[i].lstrip(' ')
        mobil[i] = mobil[i].rstrip(' ')
        if re.search(pattern2, mobil[i]):
            mobil[i] = re.sub(pattern2, '', mobil[i])
        mobil[i] = ''.join([x for x in mobil[i] if x not in '()-'])  
            

for i, j in enumerate(mobil):
    if type(j) == str:
        if len(j) == 7:
            mobil[i] = '7812' + mobil[i]
        elif len(j) == 11 and j[0] == '8':
            mobil[i] = '7' + j[1:]
        elif len(j) == 10:
            mobil[i] = '7' + j
            
            
taxi['Телефон'] = mobil   

taxi = taxi.drop_duplicates(subset='Телефон').reset_index(drop=True)

taxi

,Телефон,Источник
0,79523658340,Заявки 2 кв
1,79841165259,Заявки 2 кв
2,79523525824,Заявки 2 кв
3,79213598313,Заявки 2 кв
4,79046426712,Заявки 2 кв
...,...,...
887,78125115181,Звонки 2 кв
888,79217523147,Звонки 2 кв
889,78126988344,Звонки 2 кв
890,79112696480,Звонки 2 кв


In [6]:
vu_telefons = pd.read_excel(r'C:\Users\AmanovRA\Текущее\Такси_2й_квартал\VU_taxi.xlsx')

phones = vu_telefons['Телефон'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for i, j in enumerate(phones):
    if len([x for x in j if x.isdigit()]) == 10:
        phones[i] = '7' + ''.join([x for x in j if x.isdigit()])    
    if len([x for x in j if x.isdigit()]) == 11:
        temp_arr = ''.join([x for x in j if x.isdigit()])
        if temp_arr[0] == '8':
            phones[i] = '7' + temp_arr[1:]
        elif temp_arr[0] == '7':    
            phones[i] = temp_arr

for i, j in enumerate(phones):
    if re.search(pattern, j):
        if '(61)' in j:
            phones[i] = phones[i].replace('(61)', ', 321-61-61, ')
        elif  '(39)' in j:   
            phones[i] = phones[i].replace('(39)', ', 305-73-39, ')
        elif '(73)' in j:
            phones[i] = phones[i].replace('(73)', '')
            phones[i] += ' , 558-73-09 '
        elif ('21') in j:
            phones[i] = phones[i].replace('(21)', '(921)')
    phones[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
    phones[i] = phones[i].lstrip(' ')
    phones[i] = phones[i].rstrip(' ')
    if re.search(pattern2, phones[i]):
        phones[i] = re.sub(pattern2, '', phones[i])
    phones[i] = ''.join([x for x in phones[i] if x not in '()-']) 
    
phones2 = phones.copy()

for i, j in enumerate(phones2):
    phones2[i] = j.split()
    
from functools import reduce
for i, j in enumerate(phones2):
    try:
        phones2[i].append(reduce(lambda x, y: x + y, [z for z in j if len(z) < 7]))
    except:
        continue
        
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if x != len(j) - 1:
            if len(y) == 4 and len(j[x + 1]) == 7 and y[:2] in ('79', '89'):
                    phones2[i].append(y + j[x + 1])
            elif y in ('7', '8') and len(j[x + 1]) == 10:
                phones2[i].append(y + j[x + 1])
                
                
for i, j in enumerate(phones2):
    phones2[i] = [x for x in phones2[i] if len(x) >= 7]
    
    
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if len(y) == 7:
            phones2[i][x] = '7812' + phones2[i][x]
        elif len(y) == 11 and y[0] == '8':
            phones2[i][x] = '7' + y[1:]
        elif len(y) == 10:
            phones2[i][x] = '7' + y
            
def concat(lst):
    arr = []
    for i in lst:
        arr.append(' '.join(i))
    return arr  

phones2 = concat(phones2)
            
vu_telefons['Чистые_телефоны'] = phones2            

In [7]:
lst_search = vu_telefons['Чистые_телефоны'].to_list()

lst_with_coincidenc_telefon = []
lst_with_coincidenc_source = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc_telefon.append([])
    lst_with_coincidenc_source.append([])
    for i, j in enumerate(taxi['Телефон']):
        if str(j) in row:
            lst_with_coincidenc_telefon[index].append(str(j))
            lst_with_coincidenc_source[index].append(taxi['Источник'][i])
            break
            
vu_telefons['Такси-телефон'], vu_telefons['Такси-источник'] = lst_with_coincidenc_telefon, lst_with_coincidenc_source           
            
def join_list(lst):
    return ', '.join([str(i) for i in lst])

vu_telefons['Такси-телефон'], vu_telefons['Такси-источник'] = vu_telefons['Такси-телефон'].apply(join_list), vu_telefons['Такси-источник'].apply(join_list)

vu_telefons.to_excel('Valere_taxi.xlsx')